In [ ]:
import pandas as pd
import openai

In diesem Notebook wird aus dem Dataset base_keywords_sentimement mit entsprechenden multimodalen Deep-Fakes ergänzt


In [ ]:
df = pd.read_csv('base_keywords_sentiment_reduced.csv')

print(df.head())
# print the keywords_only for the first 20 rows

# add a new column which states the lengtth in words in column text
df['text_length'] = df['text'].apply(lambda x: len(x.split()))

# print the text_length, text and keywords_only for the first 20 rows
df[['text_length', 'text', 'keywords_only']].head(20)

## Text:  GPT 3.5-Turbo

In [69]:
with open("gpt3.5_api.txt", "r") as f:
    api_key = f.read().strip()

class chatGPT: 
    def __init__(self, api_key, rolle):
        openai.api_key = api_key
        self.dialog = [{"role":"system", "content":rolle}]

    def question(self, frage):
        self.dialog.append({"role":"user", "content":frage})
        ergebnis = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=self.dialog)
        antwort = ergebnis["choices"][0].message.content
        self.dialog.append({"role":"assistant", "content":antwort})
        return antwort

In [70]:
role = "In stlye of a google user that writes a review about a place."
prompt = "Write a review about the Italien Restaurant about some of the keywords:"
chatGPT(api_key, role).question(prompt)

print(chatGPT(api_key, role).question(prompt))

I recently visited the Italian Restaurant in Texas and I have to say, it was an amazing experience! From the moment I walked in, the atmosphere was warm and welcoming with beautiful decor and lively music. The staff were incredibly friendly and knowledgeable about the menu, and they went out of their way to make us feel comfortable.

As for the food, it was absolutely outstanding. I ordered the spaghetti carbonara and it was hands-down the best I've ever had. The noodles were perfectly cooked and the sauce was the perfect combination of creamy and savory. My companion ordered the veal Milanese and was equally impressed - the meat was tender and flavorful, and the portion size was generous.

To top it off, the wine list was impressive and featured a broad selection of Italian and American wines. Our server recommended a lovely Chianti that perfectly complemented our meal.

Overall, I would highly recommend this Italian Restaurant to anyone looking for an exceptional dining experience. T

## Image: Dall - E - 2


In [ ]:
PROMPT = "Photo of a nice experience at Bowling alley, photography, high quality, high resolution, 4k"

with open("dalleapi.txt", "r") as f:
    openai.api_key = f.read()

response = openai.Image.create(
    prompt=PROMPT,
    n=2,
    size="256x256",
)

# write the url to the image to the file: 02_Visualizations\dalle-2-tries.txt as new line
print(response["data"][0]["url"])
#save the image to the file: 02_Visualizations\dalle-2-tries.txt as new line in txt file

with open("02_Visualizations\dalle-2-tries.txt", "a") as f:
    f.write("\nPROMPT " + PROMPT +": " + response["data"][0]["url"] + " ")

## Tries with StableDiffusion aka Dreamstudio

In [ ]:
import os
import io
import warnings
from PIL import Image
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

# Our Host URL should not be prepended with "https" nor should it have a trailing slash.
os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'

# read the api key from draeamsudio.txt

with open('dreamstudio.txt', 'r') as f:
    api_key = f.read()

os.environ['STABILITY_KEY'] = api_key

In [ ]:
stability_api = client.StabilityInference(
    key=os.environ['STABILITY_KEY'], # API Key reference.
    verbose=True, # Print debug messages.
    engine="stable-diffusion-512-v2-1", # Set the engine to use for generation.
    # Available engines: stable-diffusion-v1 stable-diffusion-v1-5 stable-diffusion-512-v2-0 stable-diffusion-768-v2-0
    # stable-diffusion-512-v2-1 stable-diffusion-768-v2-1 stable-inpainting-v1-0 stable-inpainting-512-v2-0
)

In [ ]:
# Set up our initial generation parameters.
answers = stability_api.generate(
    prompt="Bad experience at italien restaurant,high quality photography, 4k, close-up",
    #seed=992446758, # If a seed is provided, the resulting generated image will be deterministic.
                    # What this means is that as long as all generation parameters remain the same, you can always recall the same image simply by generating it again.
                    # Note: This isn't quite the case for Clip Guided generations, which we'll tackle in a future example notebook.
    steps=30, # Amount of inference steps performed on image generation. Defaults to 30.
    cfg_scale=7.0, # Influences how strongly your generation is guided to match your prompt.
                   # Setting this value higher increases the strength in which it tries to match your prompt.
                   # Defaults to 7.0 if not specified.
    width=512, # Generation width, defaults to 512 if not included.
    height=512, # Generation height, defaults to 512 if not included.
    samples=2, # Number of images to generate, defaults to 1 if not included.
    sampler=generation.SAMPLER_K_DPMPP_2M # Choose which sampler we want to denoise our generation with.
                                                 # Defaults to k_dpmpp_2m if not specified. Clip Guidance only supports ancestral samplers.
                                                 # (Available Samplers: ddim, plms, k_euler, k_euler_ancestral, k_heun, k_dpm_2, k_dpm_2_ancestral, k_dpmpp_2s_ancestral, k_lms, k_dpmpp_2m)
)

# Set up our warning to print to the console if the adult content classifier is tripped.
# If adult content classifier is not tripped, save generated images.
for resp in answers:
    for artifact in resp.artifacts:
        if artifact.finish_reason == generation.FILTER:
            warnings.warn(
                "Your request activated the API's safety filters and could not be processed."
                "Please modify the prompt and try again.")
        if artifact.type == generation.ARTIFACT_IMAGE:
            img = Image.open(io.BytesIO(artifact.binary))
            img.save(str(artifact.seed)+ ".png") # Save our generated images with their seed number as the filename.